<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto NLP

In [124]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
import pandas as pd
import numpy as np
import string 
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Definimos el Corpus

¿Que es el Corpus?

Base de conocimiento, indica las preguntas frecuentes que podría preguntar el cliente

In [129]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
f=open("/content/drive/My Drive/Prueba_ChatBot/corpus.txt",'r',errors='ignore')
raw=f.read()

Imprimimos la lectura del **Corpus (.txt)** en modo texto plano.

Los puntos, signos de interrogación y caracteres espaciados serán los que utilicemos a la hora de partir las frases a devolver para el cliente.

In [151]:
raw

'sitemas se encuentra en el piso 7.\n\npuedes levantar una peticion llamando a la extension 561-221.\n\nel equipo de data analytics esta en cabezado por alberto.\n\nel hoario de atencin es de 8am-6:30pm\n'

# Pre procesamiento del texto **Corpus**



In [133]:
raw=raw.lower() #Convertimos a minúscula

In [152]:
raw

'sitemas se encuentra en el piso 7.\n\npuedes levantar una peticion llamando a la extension 561-221.\n\nel equipo de data analytics esta en cabezado por alberto.\n\nel hoario de atencin es de 8am-6:30pm\n'

In [154]:
#Tokenizamos en base a sentencias
sent_tokens = nltk.sent_tokenize(raw)
sent_tokens
sent_tokens[1]

'el hoario de atencin es de 8am-6:30pm'

In [136]:
#Tokenizamos en base a palabras
word_tokens = nltk.word_tokenize(raw)
word_tokens

['sitemas',
 'se',
 'encuentra',
 'en',
 'el',
 'piso',
 '7.',
 'puedes',
 'levantar',
 'una',
 'peticion',
 'llamando',
 'a',
 'la',
 'extension',
 '561-221.',
 'el',
 'equipo',
 'de',
 'data',
 'analytics',
 'esta',
 'en',
 'cabezado',
 'por',
 'alberto',
 '.',
 'el',
 'hoario',
 'de',
 'atencin',
 'es',
 'de',
 '8am-6:30pm']

In [137]:
#Lematizador
lemmer=nltk.stem.WordNetLemmatizer()

#Función para lematizar uno a uno los wordtokens
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]

In [138]:
#Removemos signos de puntuación
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation)

#Se le da un texto a la función y está le elimina los signos de puntuación y la tokeniza
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Pre procesamiento del texto (Mensaje de usuario) 

#+

#similitud del texto del usuario vs Corpus




**CONSIDERACIONES:**

A menudo dentro de un texto suelen dominar los **artículos** y **preposiciones**, por lo que todas esas palabras que no contienen la información base, se les dará un menor peso.

<img src="https://seaprueba.com/wp-content/uploads/2019/09/Tipos-de-Art%C3%ADculos.png">

<img src="https://i.pinimg.com/originals/66/6b/e9/666be9803c3b850a02bec4f811bb7cc4.jpg">

Para esto usaremos la siguiente función:

**TF-IDF:** Pondera las palabras que no aportan información relevante con un menor peso, tales como preposiciones o artículos que tienden a repetirse mucho. El resultado tras aplicarlo a un texto es un vector que indica la palabra o la oración.

##¿Como evaluamos la similitud entre el mensaje del usuario y el Corpus?

**Similitud de coseno:** Medida de similitud entre dos vectores distintos de cero. Se usará para encontrar similitud entre las palabras ingresadas por el usuario y las palabras del Corpus




In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [140]:
#Función para determinar la similitud del texto insetado y el Corpus

#Definimos función que como entrada tendrá el mensaje del usuario
def response (user_response):
  #Definimos la respuesta como vacía
  robo_response=''
  #Añadimos la respuesta del usuario al Corpus
  sent_tokens.append(user_response)
  #Como tokenizador utilizaremos la lematización que definimos e indicamos que el idioma es español
  TfidfVec= TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('spanish'))
  #Ajustamos en base al listado del sent_tokens (oraciones del corpus como mensaje de usuario)
  tfidf = TfidfVec.fit_transform(sent_tokens)

  #Evaluamos la similitud de coseno entre mensaje de usuario (tfidf[-1]) y el CORPUS (tfidf)

  #Definimos función que no arrojara el resultado más aproximado entre el mensaje de usuario y el Corpus
  vals = cosine_similarity(tfidf[-1],tfidf) #Similitud entre vector de entrada (usuario) vs vector(Corpus)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]

  if(req_tfidf==0):
    #Si el resultado es cero, dira "lo siento"
    robo_response = robo_response+"Lo siento, no te he entendido. Si no puedo responder a lo que bucas contacta con Sistemas"
    return robo_response

  else:
    #En otro caso, nos devolvera la sentencia de tokens con el indice en donde obtuve la mayor similitud de coseno
    robo_response = robo_response+sent_tokens[idx]
    return robo_response





# Coincidencias Manuales

In [141]:
SALUDOS_INPUTS = ("hola","saludos","qué tal","buenos días")
SALUDOS_OUTPUTS = ("Hola","Hola, ¿Cómo te puedo ayudar?","Hola, encantado de hablar contigo","Hola, ¿Qué tal?")

#Definimos una función que como entrada tendrá un saludo
def saludos(sentence):
  #Cortamos la sentencia
  for word in sentence.split():
    #Convertimos todo a minusculas para poder hacer match con nuestros posibles inputs
    if word.lower() in SALUDOS_INPUTS:
      #Regresa un saludo tomado aleatoriamente
      return random.choice(SALUDOS_OUTPUTS)


#Generación de respuesta

In [156]:
flag = True
print("ROBOT: Mi nombre es ROBOT. Trataré de responder tus dudas, Si quieres salir, escribe 'salir")
while(flag==True):
  user_response = input()
  #Convertimos la entrada en minuscula para poder hacer match
  user_response = user_response.lower() 

  if(user_response!='salir'):

      if(user_response=='gracias' or user_response=='muchas gracias'):
          flag=True
          print("ROBOT: No hay de qué")

      else:
          if(saludos(user_response)!=None): #Si la palabra insertada por el usuario es un saludo(coincidencias manuales definidas)
            print("ROBOT: "+saludos(user_response))

          else: #Si la palabra insertada no es un saludo----> Query
              print("ROBOT: ",end="")
              print(response(user_response))
              sent_tokens.remove(user_response) #Eliminamos del corpus la respuesta del usuario

              

  else:
      flag = False
      print("Robot: Nos vemos pronto, ¡Cuidate!")


ROBOT: Mi nombre es ROBOT. Trataré de responder tus dudas, Si quieres salir, escribe 'salir
puto
ROBOT: Lo siento, no te he entendido. Si no puedo responder a lo que bucas contacta con Sistemas
hola
ROBOT: Hola, encantado de hablar contigo
salir
Robot: Nos vemos pronto, ¡Cuidate!
